In [0]:
import requests
import lxml.html as lh
import pandas as pd
from google.colab import files
import numpy as np

# Other characters
url = "https://en.wikipedia.org/wiki/List_of_Game_of_Thrones_characters"
page = requests.get(url)
doc = lh.fromstring(page.content)
characters = doc.xpath('//div[@class = "div-col columns column-width"]//ul//li')


In [0]:
#Create a dataframe to store scraped data about OTHER characters
#Step 1 - Extarct list of characters from Wikipedia 

df_MSChars = pd.DataFrame(columns = ['Name', 'Google search query', 'GOT Fandom Wiki Link', 'Season(s)', 'First seen', 'FS_Episode_URL', 'FS_Episode_AirTime', 'Appeared in', 'Also known as', 'Titles', 'Origin', 'Status', 'Death', 'Death shown in','DS_Episode_URL', 'DS_Episode_AirTime', 'Allegiance', 'Culture', 'Religion', 'Portrayed by'], index = range(len(characters)))
j = 0

for i in range(len(characters)):
  if characters[i].text_content().rsplit("as ", 1)[1] : 
    df_MSChars.loc[j]['Name'] = characters[i].text_content().rsplit("as ", 1)[1]
    print(characters[i].text_content().rsplit("as ", 1)[1])
    j+=1

df_MSChars.dropna(how= 'all', inplace = True)    
print(df_MSChars.head(5), df_MSChars.tail(5))

In [0]:
#Step 2 - Generate google search query URLs and extarct clean GOT Fandom Wiki links
print(j)
for i in range(j):
  SearchQ = df_MSChars.loc[i]['Name']+ str(" Game of thrones") # Generate search string by removing space and " characters from name and add GOT at the end of 
  googleSearch = "https://www.google.com/search?q={}".format(SearchQ)
  
  df_MSChars.loc[i]['Google search query'] = googleSearch
  page = requests.get(googleSearch)
  doc = lh.fromstring(page.content)
  
  wikilinks = doc.xpath('//a[contains(@href,"https://gameofthrones.fandom.com/wiki/")]/@href')
  
  #Clean page urls returned in google search result
  import re   

  to_remove = []
  clean_links = []
  for x, l in enumerate(wikilinks):
    clean = re.search('\/url\?q\=(.*)\&sa',l)

    # Anything that doesn't fit the above pattern will be removed
    if clean is None:
        to_remove.append(x)
        continue
    clean_links.append(clean.group(1))
   
  if len(clean_links) : df_MSChars.loc[i]['GOT Fandom Wiki Link'] = clean_links[0] 

In [0]:
#df_MSChars = df_MSChars.dropna(how = 'all') # Remove null valued rows scraped from Wikipedia page 
print(len(df_MSChars[df_MSChars['GOT Fandom Wiki Link'].isna()]))
print(df_MSChars[df_MSChars['GOT Fandom Wiki Link'].isna()])

In [0]:
# Step 3 - Scrape data from GOT Fandom Wiki pages
j = 0

for a in range(len(df_MSChars.index)) :
  if df_MSChars.loc[a]['GOT Fandom Wiki Link'] :
    page = requests.get(df_MSChars.loc[a]['GOT Fandom Wiki Link'])
    doc = lh.fromstring(page.content)
    data = doc.xpath('//div[@class = "pi-item pi-data pi-item-spacing pi-border-color"]')
  
    for i in range(len(data)):
      if "Season(s)" in data[i].text_content() :
        df_MSChars.loc[j]['Season(s)'] = data[i].text_content().rsplit('\t', 1)[1].replace("\n", "")
    
      if "First seen" in data[i].text_content() :
        df_MSChars.loc[j]['First seen'] = data[i].text_content().rsplit('\t', 1)[1].replace("\n", "")
  
        #First seen episode crawling
        FS_Episode_URL = doc.xpath('//div[@class = "pi-item pi-data pi-item-spacing pi-border-color" and contains(string(), "First seen")]//a/@href')
        if len(DS_Episode_URL) :
          df_MSChars.loc[j]['FS_Episode_URL'] = str("https://gameofthrones.fandom.com") + FS_Episode_URL[0]
  
          fse = df_MSChars.loc[j]['FS_Episode_URL']
          fse_page = requests.get(fse)
          fse_doc = lh.fromstring(fse_page.content)
          fse_x = fse_doc.xpath('//div[@class = "pi-item pi-data pi-item-spacing pi-border-color" and contains(string(), "Air date")]')
      
          if len(fse_x) :
            df_MSChars.loc[j]["FS_Episode_AirTime"] = fse_x[0].text_content().rsplit('\t', 1)[1].replace("\n", "")
  
      if "Appeared in" in data[i].text_content() :
        df_MSChars.loc[j]['Appeared in'] = data[i].text_content().rsplit('\t', 1)[1].replace("\n", "")
    
      if "Also known as" in data[i].text_content() :
        df_MSChars.loc[j]['Also known as'] = data[i].text_content().rsplit('\t', 1)[1].replace("\n", "")
    
      if "Titles" in data[i].text_content() :
        df_MSChars.loc[j]['Titles'] = data[i].text_content().rsplit('\t', 1)[1].replace("\n", "")
    
      if "Origin" in data[i].text_content() :
        df_MSChars.loc[j]['Origin'] = data[i].text_content().rsplit('\t', 1)[1].replace(")", "), ").replace("\n", "")
    
      if "Status" in data[i].text_content() :
        df_MSChars.loc[j]['Status'] = data[i].text_content().rsplit('\t', 1)[1].replace("\n", "")
    
      if "Death" in data[i].text_content() and "Death shown in" not in data[i].text_content() :
        df_MSChars.loc[j]['Death'] = data[i].text_content().rsplit('\t', 1)[1].replace("\n", "")
    
      if "Death shown in" in data[i].text_content() :
        df_MSChars.loc[j]['Death shown in'] = data[i].text_content().rsplit('\t', 1)[1].replace("\n", "")
    
        #Death shown episode crawling
        DS_Episode_URL = doc.xpath('//div[@class = "pi-item pi-data pi-item-spacing pi-border-color" and contains(string(), "Death shown in episode")]//a/@href')
        if len(DS_Episode_URL) : 
          df_MSChars.loc[j]['DS_Episode_URL'] = str("https://gameofthrones.fandom.com") + DS_Episode_URL[0]
  
          dse = df_MSChars.loc[j]['DS_Episode_URL']
          dse_page = requests.get(dse)
          dse_doc = lh.fromstring(dse_page.content)
          dse_x = dse_doc.xpath('//div[@class = "pi-item pi-data pi-item-spacing pi-border-color" and contains(string(), "Air date")]')
      
          if len(dse_x) :
            df_MSChars.loc[j]["DS_Episode_AirTime"] = dse_x[0].text_content().rsplit('\t', 1)[1].replace("\n", "")
  
      if "Allegiance" in data[i].text_content() :
        df_MSChars.loc[j]['Allegiance'] = data[i].text_content().rsplit('\t', 1)[1].replace(")", "), ").replace("\n", "")
    
      if "Culture" in data[i].text_content() :
        df_MSChars.loc[j]['Culture'] = data[i].text_content().rsplit('\t', 1)[1].replace("\n", "")
    
      if "Religion" in data[i].text_content() :
        df_MSChars.loc[j]['Religion'] = data[i].text_content().rsplit('\t', 1)[1].replace("\n", "")
    
      if "Portrayed by" in data[i].text_content() :
        df_MSChars.loc[j]['Portrayed by'] = data[i].text_content().rsplit('\t', 1)[1].replace(")", "), ").replace("\n", "")
   
    j+=1  


In [0]:
print(df_MSChars.head(5))
#Copy the df into a CSV file and export it on the device
df_MSChars.to_csv('GOT_Others.csv', index = False)
files.download('GOT_Others.csv')